In [1]:
import pandas as pd
import requests

# all postseason data 1885-2021
tables = pd.read_html(f'https://www.baseball-reference.com/postseason/')
df_postseason = tables[0]

In [2]:
# drop any columns that have missing data
df_postseason = df_postseason.dropna(axis=0, how='any')
df_postseason = df_postseason.dropna(axis=1, how='any')

In [3]:
#  get rid of future seasons
df_postseason = df_postseason.iloc[:-4, :]

# get rid of years where postseason was not played
df_postseason = df_postseason[df_postseason["Unnamed: 2"].str.contains("No Postseason played this year.")==False]

In [4]:
# split the series column into year and series name

# split the data
new = df_postseason['Series'].str.split(" ", n = 1, expand = True)

# drop the old series column
df_postseason2 = df_postseason.drop(columns = ['Series'])

# add the split columns to the dataframe
df_postseason2 = pd.concat([new, df_postseason2], axis=1)

# rename the columns
dict_rename = {0: 'Season',
               1: 'Series',
              'Unnamed: 1': 'Final_Score',
              'Unnamed: 2': 'Teams_Played'}
df_postseason2 = df_postseason2.rename(dict_rename, axis = 1)

# update the seasons to integers
df_postseason2['Season'] = df_postseason2['Season'].astype(int)
df_postseason2.head()

,Season,Series,Final_Score,Teams_Played
0,2021,World Series,4-2,"Atlanta Braves (88-73, NL) vs. Houston Astros ..."
1,2021,ALCS,4-2,"Houston Astros (95-67, AL) vs. Boston Red Sox*..."
2,2021,NLCS,4-2,"Atlanta Braves (88-73, NL) vs. Los Angeles Dod..."
3,2021,ALDS1,3-1,"Houston Astros (95-67, AL) vs. Chicago White S..."
4,2021,ALDS2,3-1,"Boston Red Sox* (92-70, AL) vs. Tampa Bay Rays..."


In [5]:
new3 = df_postseason2['Teams_Played'].str.split('(')

final = []

for row in new3:
    win_team = row[0].strip()
    
    lose_team = row[1].split('.', 1)[1].strip()
    
    updated_row = [win_team, lose_team]
    final.append(updated_row)

In [6]:
# drop the teams_played column
df_postseason3 = df_postseason2.drop(columns = ['Teams_Played'])

# add the split columns to the dataframe
win_lose_teams = pd.DataFrame(final)
df_postseason3 = pd.concat([df_postseason3, win_lose_teams], axis=1)

# rename the columns
dict_rename2 = {0: 'W_Team',
               1: 'L_Team'}
df_postseason3 = df_postseason3.rename(dict_rename2, axis=1)

In [7]:
# for the Winning_Team and Losing_Team columns, update the teams

def clean_team(team_str):
    """ clean the team string
        
    Args:
        team_str (str): the initial string representing the team
        
    Returns:
        team_name (str): the cleaned string
    """
    
    team_list = str(team_str).split('(')
    
    clean_team = team_list[0]
    
    clean_team = clean_team.strip()
    
    if clean_team.endswith('*'):
        clean_team = clean_team[:-1]
    
    return clean_team

In [8]:
df_postseason3['Winning_Team'] = df_postseason3['W_Team'].map(clean_team)
df_postseason3['Losing_Team'] = df_postseason3['L_Team'].map(clean_team)

In [9]:
# drop the old winning and losing team columns
df_postseason4 = df_postseason3.drop(columns = ['W_Team', 'L_Team'])

In [10]:
df_postseason4 = df_postseason4.sort_values('Season', ascending=False)

In [11]:
# drop all of the rows before 1988 and after 2018
df_filtered1 = df_postseason4[df_postseason4['Season'] <= 2018] 
df_filtered2 = df_postseason4[df_postseason4['Season'] >= 1988]

df_postseason_final = pd.merge(df_filtered1, df_filtered2, how="inner")

df_postseason_final.head()

,Season,Series,Final_Score,Winning_Team,Losing_Team
0,2018.0,NLWC,1-0,Colorado Rockies,Chicago Cubs
1,2018.0,ALWC,1-0,New York Yankees,Oakland Athletics
2,2018.0,NLDS2,3-1,Los Angeles Dodgers,Atlanta Braves
3,2018.0,NLDS1,3-0,Milwaukee Brewers,Colorado Rockies
4,2018.0,ALDS2,3-0,Houston Astros,Cleveland Indians


In [12]:
def get_league(series_str):
    """ determine the league from the series string
        
    Args:
        series_str (str): the series string
        
    Returns:
        league (str): the league of the series
    """
    if str(series_str).startswith('AL'):
        league = 'AL'
    elif str(series_str).startswith('NL'):
        league = 'NL'
    else:
        league = 'None'

    return league

In [13]:
df_postseason_final['League'] = df_postseason_final['Series'].map(get_league)

# reorder columns
df_postseason_final = df_postseason_final[["Season", "League", "Series", 
                                           'Final_Score', 'Winning_Team', 'Losing_Team']]
df_postseason_final.head()

,Season,League,Series,Final_Score,Winning_Team,Losing_Team
0,2018.0,NL,NLWC,1-0,Colorado Rockies,Chicago Cubs
1,2018.0,AL,ALWC,1-0,New York Yankees,Oakland Athletics
2,2018.0,NL,NLDS2,3-1,Los Angeles Dodgers,Atlanta Braves
3,2018.0,NL,NLDS1,3-0,Milwaukee Brewers,Colorado Rockies
4,2018.0,AL,ALDS2,3-0,Houston Astros,Cleveland Indians


In [14]:
# get the series played
def get_series(series_str):
    """ determine the series from the series string
        
    Args:
        series_str (str): the series string
        
    Returns:
        series (str): the series played
    """
    if str(series_str).startswith('AL'):
        series = series_str[len('AL'):]
    elif str(series_str).startswith('NL'):
        series = series_str[len('NL'):]
    else:
        series = 'World Series'
        
        
    # Championship Series, Division Series 1 and 2, Wild Card, World Series
    if series == 'CS':
        series = 'Championship'
    elif series == 'DS1':
        series = 'Division1'
    elif series == 'DS2':
        series = 'Division2'
    elif series == 'WC':
        series = 'Wildcard'
    else:
        series = 'WorldSeries'

    return series

In [15]:
df_postseason_final['Series_Played'] = df_postseason_final['Series'].map(get_series)

In [16]:
# reorder columns
df_postseason_final = df_postseason_final[["Season", "League", "Series_Played", 
                                           'Final_Score', 'Winning_Team', 'Losing_Team']]
df_postseason_final.head()

,Season,League,Series_Played,Final_Score,Winning_Team,Losing_Team
0,2018.0,NL,Wildcard,1-0,Colorado Rockies,Chicago Cubs
1,2018.0,AL,Wildcard,1-0,New York Yankees,Oakland Athletics
2,2018.0,NL,Division2,3-1,Los Angeles Dodgers,Atlanta Braves
3,2018.0,NL,Division1,3-0,Milwaukee Brewers,Colorado Rockies
4,2018.0,AL,Division2,3-0,Houston Astros,Cleveland Indians


In [17]:
# determine if the winning team is in the AL or NL
def determine_league(winning_team):
    """ determine if the winning team is in the AL (true) or NL (false)
    
    Args:
        winnning_team (str) : the winning team, as a string
    
    Returns:
        in_al (Boolean) : True if the team is in the AL, or False if the team is in the NL
    """
    
    AL_teams = ['Baltimore Orioles', 'Boston Red Sox','New York Yankees', 'Tampa Bay Rays',
                'Toronto Blue Jays', 'Chicago White Sox', 'Cleveland Indians', 'Detroit Tigers',
                'Kansas City Royals', 'Minnesota Twins', 'Houston Astros', 'Los Angeles Angels',
                'Oakland Athletics', 'Seattle Mariners', 'Texas Rangers']
    
    return winning_team in AL_teams

In [18]:
df_postseason_final['AL_Win'] = df_postseason_final['Winning_Team'].map(determine_league)
df_postseason_final.head()

,Season,League,Series_Played,Final_Score,Winning_Team,Losing_Team,AL_Win
0,2018.0,NL,Wildcard,1-0,Colorado Rockies,Chicago Cubs,False
1,2018.0,AL,Wildcard,1-0,New York Yankees,Oakland Athletics,True
2,2018.0,NL,Division2,3-1,Los Angeles Dodgers,Atlanta Braves,False
3,2018.0,NL,Division1,3-0,Milwaukee Brewers,Colorado Rockies,False
4,2018.0,AL,Division2,3-0,Houston Astros,Cleveland Indians,True


Need to do:
- constrain years from 2012-2018
- make sure that all Winning_Team and Losing_Team names are the same as the MLB data
- Add columns winning_team_abbr and losing_team_abbr

In [19]:
# constrain years from 2012-2018

# initialize empty dataframe
df_postseason_constrained = pd.DataFrame()

# list of years
list_years = [2018.0, 2017.0, 2016.0, 2015.0, 2014.0, 2013.0, 2012.0]

for year in list_years:
    s_bool = df_postseason_final['Season'] == year
    df_temp = df_postseason_final.loc[s_bool, :]
    
    df_postseason_constrained = df_postseason_constrained.append(df_temp)

df_postseason_constrained

,Season,League,Series_Played,Final_Score,Winning_Team,Losing_Team,AL_Win
0,2018.0,NL,Wildcard,1-0,Colorado Rockies,Chicago Cubs,False
1,2018.0,AL,Wildcard,1-0,New York Yankees,Oakland Athletics,True
2,2018.0,NL,Division2,3-1,Los Angeles Dodgers,Atlanta Braves,False
3,2018.0,NL,Division1,3-0,Milwaukee Brewers,Colorado Rockies,False
4,2018.0,AL,Division2,3-0,Houston Astros,Cleveland Indians,True
...,...,...,...,...,...,...,...
58,2012.0,AL,Division2,3-2,Detroit Tigers,Oakland Athletics,True
59,2012.0,NL,Division1,3-2,St. Louis Cardinals,Washington Nationals,False
60,2012.0,NL,Division2,3-2,San Francisco Giants,Cincinnati Reds,False
61,2012.0,AL,Wildcard,1-0,Baltimore Orioles,Texas Rangers,True


In [20]:
# make sure that all team names are the same as the names in the MLB cleaned data
df_postseason_constrained = df_postseason_constrained.replace('Los Angeles Angels of Anaheim', 'Los Angeles Angels')

In [21]:
# add winning_team_abbr column
def team_abbr(team_str):
    """ get the abbreviation for the team name
    
    Args:
        team_str (str): the string representing the team name
        
    Results:
        team_abbr (str): the abbreviation representing the team name
    """
    
    abbr_dict = {
        'Arizona Diamondbacks' : 'ARI',
        'Atlanta Braves' : 'ATL',
        'Baltimore Orioles' : 'BAL',
        'Boston Red Sox' : 'BOS',
        'Chicago Cubs' : 'CHC',
        'Chicago White Sox' : 'CHW',
        'Cincinnati Reds' : 'CIN',
        'Cleveland Indians' : 'CLE',
        'Colorado Rockies' : 'COL',
        'Detroit Tigers' : 'DET',
        'Houston Astros' : 'HOU',
        'Kansas City Royals' : 'KCR',
        'Los Angeles Angels' : 'LAA',
        'Los Angeles Dodgers' : 'LAD',
        'Miami Marlins' : 'MIA',
        'Milwaukee Brewers' : 'MIL',
        'Minnesota Twins' : 'MIN',
        'New York Mets' : 'NYM',
        'New York Yankees' : 'NYY',
        'Oakland Athletics' : 'OAK',
        'Philadelphia Phillies' : 'PHI',
        'Pittsburgh Pirates' : 'PIT',
        'San Diego Padres' : 'SDP',
        'Seattle Mariners' : 'SEA',
        'San Francisco Giants' : 'SFG',
        'St. Louis Cardinals' : 'STL',
        'Tampa Bay Rays' : 'TBR',
        'Texas Rangers' : 'TEX',
        'Toronto Blue Jays' : 'TOR',
        'Washington Nationals' : 'WSN' }
    
    team_abbr = abbr_dict[team_str]
    
    return team_abbr

In [22]:
# map the team abbreviations into new columns
df_postseason_constrained['Winning_Team_Abbr'] = df_postseason_constrained['Winning_Team'].map(team_abbr)
df_postseason_constrained['Losing_Team_Abbr'] = df_postseason_constrained['Losing_Team'].map(team_abbr)
df_postseason_constrained

,Season,League,Series_Played,Final_Score,Winning_Team,Losing_Team,AL_Win,Winning_Team_Abbr,Losing_Team_Abbr
0,2018.0,NL,Wildcard,1-0,Colorado Rockies,Chicago Cubs,False,COL,CHC
1,2018.0,AL,Wildcard,1-0,New York Yankees,Oakland Athletics,True,NYY,OAK
2,2018.0,NL,Division2,3-1,Los Angeles Dodgers,Atlanta Braves,False,LAD,ATL
3,2018.0,NL,Division1,3-0,Milwaukee Brewers,Colorado Rockies,False,MIL,COL
4,2018.0,AL,Division2,3-0,Houston Astros,Cleveland Indians,True,HOU,CLE
...,...,...,...,...,...,...,...,...,...
58,2012.0,AL,Division2,3-2,Detroit Tigers,Oakland Athletics,True,DET,OAK
59,2012.0,NL,Division1,3-2,St. Louis Cardinals,Washington Nationals,False,STL,WSN
60,2012.0,NL,Division2,3-2,San Francisco Giants,Cincinnati Reds,False,SFG,CIN
61,2012.0,AL,Wildcard,1-0,Baltimore Orioles,Texas Rangers,True,BAL,TEX


In [23]:
# reorder the columns
final_df_postseason = df_postseason_constrained[['Season', 'League', 'Series_Played', 'Final_Score', 
                                                 'Winning_Team', 'Winning_Team_Abbr', 
                                                 'Losing_Team', 'Losing_Team_Abbr']]
final_df_postseason

,Season,League,Series_Played,Final_Score,Winning_Team,Winning_Team_Abbr,Losing_Team,Losing_Team_Abbr
0,2018.0,NL,Wildcard,1-0,Colorado Rockies,COL,Chicago Cubs,CHC
1,2018.0,AL,Wildcard,1-0,New York Yankees,NYY,Oakland Athletics,OAK
2,2018.0,NL,Division2,3-1,Los Angeles Dodgers,LAD,Atlanta Braves,ATL
3,2018.0,NL,Division1,3-0,Milwaukee Brewers,MIL,Colorado Rockies,COL
4,2018.0,AL,Division2,3-0,Houston Astros,HOU,Cleveland Indians,CLE
...,...,...,...,...,...,...,...,...
58,2012.0,AL,Division2,3-2,Detroit Tigers,DET,Oakland Athletics,OAK
59,2012.0,NL,Division1,3-2,St. Louis Cardinals,STL,Washington Nationals,WSN
60,2012.0,NL,Division2,3-2,San Francisco Giants,SFG,Cincinnati Reds,CIN
61,2012.0,AL,Wildcard,1-0,Baltimore Orioles,BAL,Texas Rangers,TEX


In [26]:
final_df_postseason.to_csv('postseason_data.csv')